In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
from tqdm import tqdm
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException

driver = webdriver.Chrome()
driver.get("https://slmis.xu.edu.ph/")

# Getting Class Roster

In [2]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
def go_to_home_page():
    driver.get("https://slmis.xu.edu.ph/psp/ps/EMPLOYEE/HRMS/s/WEBLIB_PTPP_SC.HOMEPAGE.FieldFormula.IScript_AppHP?pt_fname=CO_EMPLOYEE_SELF_SERVICE&amp;FolderPath=PORTAL_ROOT_OBJECT.CO_EMPLOYEE_SELF_SERVICE&amp;IsFolder=true")

def check_if_loaded(by, filter):
    try:    
        element_present = EC.presence_of_element_located((by, filter))
        WebDriverWait(driver, 10).until(element_present)
    except TimeoutException:
        print("Took too long to load")
def get_faculty_schedule():
    go_to_home_page()
    #Going to faculty center
    check_if_loaded(By.XPATH, '//*[@id="ptifrmtgtframe"]')
    faculty_center = driver.find_element(By.XPATH, '//*[@id="ptifrmtgtframe"]')
    driver.switch_to.frame(faculty_center)
    faculty_center_button = driver.find_element(By.XPATH, '//*[@id="ptppscappnavtbl"]/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr/td/table/tbody/tr[2]/td/ul/li[1]/a')
    faculty_center_button.click()
    driver.switch_to.default_content()
    #Techhing schedule
    check_if_loaded(By.XPATH, '//*[@id="ptifrmtgtframe"]')
    teaching_schedule = driver.find_element(By.XPATH, '//*[@id="ptifrmtgtframe"]')
    driver.switch_to.frame(teaching_schedule)
    main_table = driver.find_element(By.XPATH,'//*[@id="INSTR_CLASS_VW$scroll$0"]/tbody/tr[1]/td/table')
    all_rows = main_table.find_elements(By.TAG_NAME, "tr")

    r = all_rows[1]
    courses_dictionary = {}
    for r in tqdm(all_rows):
        all_cells = r.find_elements(By.TAG_NAME, "td")
        #Get the name of the course

        for cell in all_cells:
            try:
                cell_element = cell.find_element(By.TAG_NAME, "span")
                if "CLASS_TIT" in cell_element.get_attribute('id'):
                    course_element = cell_element.find_element(By.TAG_NAME, "a")
                    course_name = course_element.get_attribute('innerHTML')
                    courses_dictionary[course_name] = r
            except NoSuchElementException:
                continue
    return courses_dictionary

def authentication(username, password):
    try:
        user_element = driver.find_element(By.XPATH, '//*[@id="userid"]')
        pass_element = driver.find_element(By.XPATH, '//*[@id="pwd"]')
        ActionChains(driver) \
            .click(user_element) \
            .key_down(Keys.CONTROL) \
            .send_keys("a") \
            .key_up(Keys.CONTROL) \
            .send_keys(username)\
            .click(pass_element) \
            .key_down(Keys.CONTROL) \
            .send_keys("a") \
            .key_up(Keys.CONTROL) \
            .send_keys(password)\
            .key_down(Keys.ENTER) \
            .perform()
        
        if "errorCode" in driver.current_url:
            print("Authentication Failed")
            return False
        return True
    except NoSuchElementException:
        print("Already Logged in")
    

In [3]:
authentication('gsulay', '{4_k66iI')

True

In [4]:
go_to_home_page()

In [5]:
courses_dictionary = get_faculty_schedule()

100%|██████████| 8/8 [00:00<00:00, 14.54it/s]


In [6]:
view_cleaner = lambda val: val.replace("<br>\n"," ").replace("   "," ")
for i in courses_dictionary.keys():
    print(view_cleaner(i))

CE 17-A (3366)
CE 17-LA (3367)
CE 23-L B2 (1909)
CE 26.1-LB2 (1602)
ESC 12-B1 (1433)
ESC 12-B3 (1930)
ESC 12-B4 (1439)


In [7]:
raw_text = split('<br>\n')

NameError: name 'split' is not defined

In [8]:

all_sections_dict

NameError: name 'all_sections_dict' is not defined

In [9]:
val = 4
def click_gradebook(val):
    all_sections_dict = get_faculty_schedule()
    current_row = all_sections_dict[list(all_sections_dict.keys())[val]]
    [i for i in current_row.find_elements(By.TAG_NAME, "a") if 'GRADEBOOK' in i.get_attribute('id')][0].click()
go_to_home_page()
click_gradebook(4)

100%|██████████| 8/8 [00:00<00:00, 12.09it/s]


In [161]:
desc = [i.get_attribute("innerHTML") for i in driver.find_elements(By.TAG_NAME, 'span') if "DERIVED_LAM_ASSIGNMENT_DESCR" in i.get_attribute("id")]


['Prelim Ex', 'Midterm Ex', 'QZ-M', 'ATT-M', 'CP-M', 'SW-M', 'PRJ-M']

# OPEN THE GRADEBOOK BEFORE RUNNING SUCCEEDING

In [196]:
driver.switch_to.default_content()
driver.switch_to.frame(driver.find_element(By.NAME, "TargetContent"))
table = driver.find_element(By.XPATH, '//table[@role="presentation"]')
inputs = table.find_elements(By.TAG_NAME, "input")
element_names = [i.get_attribute("id") for i in inputs if "DERIVED_LAM_GRADE" in i.get_attribute("id")]
students = [i.get_attribute("innerHTML") for i in driver.find_elements(By.TAG_NAME, 'span') if "HCR_PERSON_NM_I_NAME" in i.get_attribute("id")]
desc = [i.get_attribute("innerHTML") for i in driver.find_elements(By.TAG_NAME, 'span') if "DERIVED_LAM_ASSIGNMENT_DESCR" in i.get_attribute("id")]

In [213]:
# determine if exceed limit
def get_no_pages():
    driver.switch_to.default_content()
    driver.switch_to.frame(driver.find_element(By.NAME, "TargetContent"))
    table = driver.find_element(By.XPATH, '//table[@role="presentation"]')
    page_no = 1
    while True:
        try:
            check_if_loaded(By.XPATH, '//table[@role="presentation"]')
            next_bttn_element = table.find_element(By.XPATH, '//a[@name="DERIVED_LAM_RIGHT_MOVE"]')
            next_bttn_element.click()
            time.sleep(0.5)
        except StaleElementReferenceException:
            break
        except NoSuchElementException:
            break
        page_no += 1

    return page_no
pages_no = get_no_pages()

click_gradebook(4)
all_desc = []

for i in range(pages_no):
    driver.switch_to.default_content()
    check_if_loaded(By.NAME, "TargetContent")
    driver.switch_to.frame(driver.find_element(By.NAME, "TargetContent"))
    #Going to faculty center
    time.sleep(1)
    desc = [i.get_attribute("innerHTML") for i in driver.find_elements(By.TAG_NAME, 'span') if "DERIVED_LAM_ASSIGNMENT_DESCR" in i.get_attribute("id")]
    all_desc.append(desc)

    if i != pages_no - 1:
        print("Next page")
        driver.switch_to.default_content()
        driver.switch_to.frame(driver.find_element(By.NAME, "TargetContent"))
        table = driver.find_element(By.XPATH, '//table[@role="presentation"]')
        check_if_loaded(By.XPATH, '//table[@role="presentation"]')
        next_bttn_element = table.find_element(By.XPATH, '//a[@name="DERIVED_LAM_RIGHT_MOVE"]')
        next_bttn_element.click()


100%|██████████| 8/8 [00:01<00:00,  4.68it/s]


Next page


In [14]:
all_desc = [[1,2,3,4,5,6,7],[4,5,6,7,8,9,10]]
prev = all_desc[:-2]
former, latter = all_desc[-2:]

[4, 5, 6, 7, 8, 9, 10]

In [218]:
def combine_headers()

100%|██████████| 8/8 [00:01<00:00,  4.51it/s]


In [220]:
driver.get(all_gradebook_links[2])

In [160]:
df = pd.DataFrame({'Names':students})
for i in desc:
    df[i] = ""

df.to_excel("format.xlsx", index=False)

In [12]:
saved_df = pd.read_excel("cem24_a.xlsx")
for name in tqdm(element_names):
    try:
        col, row = [int(i) for i in name[18:].split('$')]
        
        grade = saved_df.iloc[int(row), int(col)]
        if pd.isna(grade):
            continue
        ActionChains(driver) \
        .click(driver.find_element(By.ID, name)) \
        .key_down(Keys.CONTROL) \
        .send_keys("a") \
        .key_up(Keys.CONTROL) \
        .send_keys(f"{round(grade,2)}")\
        .click(driver.find_element(By.ID, name)) \
        .key_down(Keys.CONTROL) \
        .send_keys("a") \
        .key_up(Keys.CONTROL) \
        .send_keys(f"{round(grade,2)}")\
        .perform()
        
    except StaleElementReferenceException:
        print("StaleElementReferenceException")
        time.sleep(1)
        ActionChains(driver) \
        .click(driver.find_element(By.ID, name)) \
        .key_down(Keys.CONTROL) \
        .send_keys("a") \
        .key_up(Keys.CONTROL) \
        .send_keys(f"{round(grade,2)}")\
        .click(driver.find_element(By.ID, name)) \
        .key_down(Keys.CONTROL) \
        .send_keys("a") \
        .key_up(Keys.CONTROL) \
        .send_keys(f"{round(grade,2)}")\
        .perform()

print("Successfully imported all final grades.\n Please click on save")

    # action = ActionBuilder(driver)
    # action.pointer_action.move_to_location(8, 0)
    # action.perform()

  0%|          | 0/189 [00:00<?, ?it/s]

100%|██████████| 189/189 [01:09<00:00,  2.72it/s]

Successfully imported all final grades.
 Please click on save


In [2]:
len("DERIVED_LAM_GRADE")

17

In [3]:
import json



{'user_name': 'gsulay', 'password': '{4_k66iI'}